In [1]:
import pandas as pd
import numpy as np

path = '../data/ETHUSDT_5m.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
init_cash = 2000
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2023-08-20 00:00:00,1668.70,1669.00,1668.48,1668.49,2160.128
1,2023-08-20 00:05:00,1668.48,1668.90,1668.15,1668.80,1651.671
2,2023-08-20 00:10:00,1668.80,1669.60,1668.02,1669.60,2537.329
3,2023-08-20 00:15:00,1669.60,1671.29,1669.25,1671.29,4070.192
4,2023-08-20 00:20:00,1671.28,1671.97,1670.51,1670.88,4347.007


In [2]:
import pandas_ta as ta
from pycaret.classification import *

#load model
model = load_model('ethusdt_5m')

#predict
df['Predict'] = predict_model(model, data=df)['prediction_label']
df['Signal'] = df['Predict']
df['ATR'] = df.ta.atr()
df['ADX'] = df.ta.adx()['ADX_14']
#help(ta.adx)

Transformation Pipeline and Model Successfully Loaded


In [3]:
df.ta.cdl_pattern(name="longline").value_counts()

CDL_LONGLINE
 0.0            15483
 100.0           1886
-100.0           1827
dtype: int64

In [4]:
help(ta.mad)

Help on function mad in module pandas_ta.statistics.mad:

mad(close, length=None, offset=None, **kwargs)
    Rolling Mean Absolute Deviation
    
    Sources:
    
    Calculation:
        Default Inputs:
            length=30
        mad = close.rolling(length).mad()
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 30
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        fillna (value, optional): pd.DataFrame.fillna(value)
        fill_method (value, optional): Type of fill method
    
    Returns:
        pd.Series: New feature generated.



In [5]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,Predict,Signal,ATR,ADX
27,2023-08-20 02:15:00,1665.09,1667.15,1665.09,1666.56,2322.404,0,0,2.169238,10.685189
28,2023-08-20 02:20:00,1666.55,1666.95,1666.45,1666.60,1032.641,0,0,2.050007,9.700960
29,2023-08-20 02:25:00,1666.60,1667.00,1665.90,1666.90,1381.222,0,0,1.982149,9.140710
30,2023-08-20 02:30:00,1666.90,1666.98,1665.42,1666.32,1475.466,0,0,1.951996,9.131816
31,2023-08-20 02:35:00,1666.33,1668.00,1666.32,1667.90,2027.793,0,0,1.932567,8.456409


In [6]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

,open,high,low,close,volume,Predict,Signal,ATR,ADX
time,,,,,,,,,
2023-08-20 02:15:00,1665.09,1667.15,1665.09,1666.56,2322.404,0,0,2.169238,10.685189
2023-08-20 02:20:00,1666.55,1666.95,1666.45,1666.60,1032.641,0,0,2.050007,9.700960
2023-08-20 02:25:00,1666.60,1667.00,1665.90,1666.90,1381.222,0,0,1.982149,9.140710
2023-08-20 02:30:00,1666.90,1666.98,1665.42,1666.32,1475.466,0,0,1.951996,9.131816
2023-08-20 02:35:00,1666.33,1668.00,1666.32,1667.90,2027.793,0,0,1.932567,8.456409


In [7]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [8]:
df['Signal'].value_counts()

0    11220
1     7949
Name: Signal, dtype: int64

In [9]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    save_size_rate = 0.125
    init_cash = init_cash
    profit_target = 500
    size_multiplier_rate = 0.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 1:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == -1:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            

bt = Backtest(df, Scalping_Strategy, cash=init_cash, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-08-20 02:15:00
End                       2023-10-25 15:35:00
Duration                     66 days 13:20:00
Exposure Time [%]                   35.786948
Equity Final [$]                    75.855036
Equity Peak [$]                   2049.095717
Return [%]                         -96.207248
Buy & Hold Return [%]                8.484543
Return (Ann.) [%]                  -99.999997
Volatility (Ann.) [%]                0.044398
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -96.298121
Avg. Drawdown [%]                  -40.982043
Max. Drawdown Duration       65 days 18:55:00
Avg. Drawdown Duration       22 days 03:15:00
# Trades                                 1187
Win Rate [%]                        37.826453
Best Trade [%]                       1.288105
Worst Trade [%]                     -0.788473
Avg. Trade [%]                    

In [10]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [11]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
                    maximize='Equity Final [$]')
stats

  0%|          | 0/12 [00:00<?, ?it/s]

CPU times: total: 3min 32s
Wall time: 6min 17s


Start                     2023-08-20 02:15:00
End                       2023-10-25 15:35:00
Duration                     66 days 13:20:00
Exposure Time [%]                    4.324691
Equity Final [$]                  8123.094629
Equity Peak [$]                  10469.044862
Return [%]                         306.154731
Buy & Hold Return [%]                8.484543
Return (Ann.) [%]               449550.346929
Volatility (Ann.) [%]          4592906.898116
Sharpe Ratio                         0.097879
Sortino Ratio                    13063.439043
Calmar Ratio                     12073.363896
Max. Drawdown [%]                  -37.234888
Avg. Drawdown [%]                   -8.804548
Max. Drawdown Duration       21 days 03:40:00
Avg. Drawdown Duration        1 days 11:32:00
# Trades                                  441
Win Rate [%]                         38.77551
Best Trade [%]                       0.777938
Worst Trade [%]                     -0.409329
Avg. Trade [%]                    

In [12]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [13]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=0.5,TPSL_rate=1.9000000000000001,size_multiplier_rate=0.30000000000000004)>

In [14]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'],name='Equity')])
#add a initial cash line
fig.add_trace(go.Scatter(x=stats['_equity_curve'].index, y=[init_cash]*len(stats['_equity_curve'].index), name='Initial Cash'))
fig.update_layout(title='Profit |'+symbol, yaxis_title='USDT', xaxis_title='Date')
fig.show()